## Tool Calling

In [83]:
import os
from dotenv import load_dotenv

load_dotenv("./../.env")

True

In [84]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

In [94]:
# Llama model
model_name = "llama3.2"
base_url = "http://localhost:11434"
llm = ChatOllama(model=model_name, base_url=base_url)

In [130]:
# Custom tools
from langchain_core.tools import tool

@tool
def add(a, b):
  """
  Add two integer numbers together
  
  Args:
  a: First integer
  b: Second integer
  """
  return a + b

@tool
def multiply(a: int, b: int) -> int:
  """
  Multiply two integer numbers together
  
  Args:
  a: First integer
  b: Second integer
  """
  return int(a) * int(b)

In [131]:
add.name, add.description, add.args_schema.schema()

('add',
 'Add two integer numbers together\n\nArgs:\na: First integer\nb: Second integer',
 {'description': 'Add two integer numbers together\n\nArgs:\na: First integer\nb: Second integer',
  'properties': {'a': {'title': 'A'}, 'b': {'title': 'B'}},
  'required': ['a', 'b'],
  'title': 'add',
  'type': 'object'})

In [132]:
add.invoke({'a': 5, 'b': 3})

8

In [133]:
multiply.invoke({"a": 103, "b": 5})

515

In [134]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

In [135]:
question = "What is 5 times 10? Also what is 6 plus 8?"
llm_with_tools.invoke(question).tool_calls

[{'name': 'multiply',
  'args': {'a': '5', 'b': '10'},
  'id': '43e998cf-0dc4-4cd7-83a7-fe622837fc58',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': '6', 'b': '8'},
  'id': '73d020b8-adf3-4f63-9a91-d049ed34892e',
  'type': 'tool_call'}]

In [136]:
question = "What is one hundred times fifteen?"
llm_with_tools.invoke(question).tool_calls

[{'name': 'multiply',
  'args': {'a': 100, 'b': 15},
  'id': '2aa373f7-f93b-437e-8607-7acf32423677',
  'type': 'tool_call'}]

### Calling tools

In [137]:
# DuckDuckGoSearch
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.invoke("Why Iran attacks Israel?")

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'In the following days, the IDF targeted additional Iranian nuclear facilities, eliminated more military commanders and nuclear scientists, and continued to work to thwart Iran\'s attacks on Israel. Why else would Israel attack Iran? Netanyahu has previously described Iran as "the head of the octopus" with "tentacles all around from the Houthis to Hezbollah to Hamas". The idea is that ... Why did Israel attack Iran now? Israel had vowed to hit back after Iran carried out a ballistic missile attack on Israel on 1 October. In that attack, Iran fired more than 180 missiles at Israel. Why Israel attacked Iran, ... Israel\'s U.N. Ambassador Danny Danon said the decision to attack Iran was an "independent decision of Israel" and an "act of self-defense." Danon went on to say the ... The latest attack, which comes just before the start of the Jewish High Holy Days, threatens to push the Middle East closer to a regionwide war.. Israel has vowed to retaliate against Iran. At the ...'

In [138]:
# TavilySearch
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
  max_results=5,
  search_depth="advanced",
  include_answer=True,
  include_raw_content=True,
)

search.invoke("Why U.S. supports Israel?")

[{'url': 'https://brusselsmorning.com/why-does-the-us-support-israel-historical-strategic-and-economic-perspectives/73082/',
  'content': 'The United States views Israel as its essential strategic partner that supports Middle East policy goals. Through the financial support and diplomatic backing the US receives from Israel, the nation maintains protective functions across the Middle Eastern region on behalf of America. The strategic alliance connects these countries through their shared military support while exchanging intelligence data and conducting trading activities. Israel operates as a dependable democratic nation that [...] US interests in the Middle East are to secure energy, resist outside influence, support Israel, combat terrorism, promote democracy, and control refugee flows. Regional dynamics heavily depend on the Israeli-Palestinian conflict, yet US policy focus on it has shifted throughout time. Israel finds support from both Christian Evangelicals and American Jews ma

In [139]:
# Wikipedia Search
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

question = "What is LLM?"

print(wikipedia.invoke(question))

Page: Large language model
Summary: A large language model (LLM) is a language model trained with self-supervised machine learning on a vast amount of text, designed for natural language processing tasks, especially language generation.
The largest and most capable LLMs are generative pretrained transformers (GPTs), which are largely used in generative chatbots such as ChatGPT, Gemini or Claude. LLMs can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information. With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information 

In [140]:
# PubMed
from langchain_community.tools.pubmed.tool import PubmedQueryRun

search = PubmedQueryRun()
print(search.invoke("What is the latest research on COVID-19?"))

No good PubMed Result was found


In [141]:
@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information.
    
    Args:
    query: The search query
    """
    
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response

@tool
def pubmed_search(query):
    """
    Search pubmed for medical and life sciences queries.
    
    Args:
    query: The search query
    """
    
    search = PubmedQueryRun()
    response = search.invoke(query)
    return response

@tool
def tavily_search(query):
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.
    
    Args:
    query: The search query
    """
    
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)
    return response

In [142]:
tools = [pubmed_search, wikipedia_search, tavily_search, multiply]
list_of_tools = { tool.name: tool for tool in tools }

In [143]:
list_of_tools

{'pubmed_search': StructuredTool(name='pubmed_search', description='Search pubmed for medical and life sciences queries.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.pubmed_search'>, func=<function pubmed_search at 0x0000021098D18900>),
 'wikipedia_search': StructuredTool(name='wikipedia_search', description='Search wikipedia for general information.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.wikipedia_search'>, func=<function wikipedia_search at 0x0000021098D1A520>),
 'tavily_search': StructuredTool(name='tavily_search', description='Search the web for realtime and latest information.\nfor examples, news, stock market, weather updates etc.\n\nArgs:\nquery: The search query', args_schema=<class 'langchain_core.utils.pydantic.tavily_search'>, func=<function tavily_search at 0x0000021098D180E0>),
 'multiply': StructuredTool(name='multiply', description='Multiply two integer numbers together\n\nArgs:\na: F

In [144]:
llm_with_tools = llm.bind_tools(tools)

In [145]:
# query = "What is the latest news?"
# query = "What is today's stock market news?"
query = "What is Python?"
# query = "How to treat lung cancer?"
# query  = "What is 2 * 3?"
response = llm_with_tools.invoke(query)
print(response.tool_calls)

[{'name': 'wikipedia_search', 'args': {'query': 'Python programming language'}, 'id': '94e5983d-d994-4ae5-990b-1f76d6de2cfb', 'type': 'tool_call'}]


### Generate result

In [ ]:
from langchain_core.messages import HumanMessage

In [157]:
# query = "What is the latest news?"
# query = "What is today's stock market news?"
# query = "What is Python?"
query = "What is medicine for lung cancer?"
# query  = "What is 2 * 3?"

messages = [HumanMessage(query)]
response = llm_with_tools.invoke(messages)

messages.append(response)

In [ ]:
for tool_call in response.tool_calls:
  name = tool_call['name'].lower()
  selected_tool = list_of_tools[name]

  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)

In [162]:
messages

[HumanMessage(content='What is medicine for lung cancer?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-28T01:43:44.2558658Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2022656700, 'load_duration': 45537600, 'prompt_eval_count': 367, 'prompt_eval_duration': 342118700, 'eval_count': 21, 'eval_duration': 1634195700, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='pubmed_search', arguments={'query': 'medicine for lung cancer'}))])}, id='run-1e2db013-9236-4d49-8655-22a10aae4219-0', tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'medicine for lung cancer'}, 'id': 'fe8d9450-186c-4f3b-b402-353d0d8f9b53', 'type': 'tool_call'}], usage_metadata={'input_tokens': 367, 'output_tokens': 21, 'total_tokens': 388}),
 ToolMessage(content='Published: 2025-06-18\nTitle: A multi-institutional randomized phase III trial o

In [163]:
response = llm_with_tools.invoke(messages)
print(response.content)

Based on the search results, the treatment for lung cancer typically involves a combination of surgical procedures, chemotherapy, radiation therapy, targeted therapy, immunotherapy, and other interventions. The specific treatment plan depends on the stage and type of lung cancer, as well as the patient's overall health.

Some common treatments for lung cancer include:

1. **Surgery**: Lobectomy (removal of the entire lobe of the lung) or segmentectomy (removal of a segment of the lung) may be recommended for early-stage lung cancer.
2. **Chemotherapy**: Chemotherapy may be used to treat lung cancer that has spread to other parts of the body.
3. **Radiation therapy**: Radiation therapy may be used to treat lung cancer, especially if it is in an early stage and cannot be removed by surgery.
4. **Targeted therapy**: Targeted therapy may be recommended for patients with specific types of lung cancer, such as non-small cell lung cancer (NSCLC) or small cell lung cancer (SCLC).
5. **Immunoth